In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Input, Flatten, Activation
from keras.preprocessing import image
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


## Training a KNN for Image Classification

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

knn = KNeighborsClassifier()

## CNN

In [3]:
inputs = Input(shape = (32,32,3))
x = Conv2D(32,3, activation = "relu", padding = "same")(inputs)
x = Conv2D(32, 3, activation = "relu", padding = "same")(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)

x = Conv2D(32, 3, activation = "relu", padding = "same")(x)
x = Conv2D(32, 3, activation = "relu", padding = "same")(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(300, activation = "relu")(x)

predictions = Dense(10, activation = "softmax")(x)

In [4]:
model = Model(inputs = inputs, outputs=predictions)

model.compile(optimizer = "rmsprop",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 32)        9248      
__________

In [5]:
model.fit(x_train, y_train, epochs = 20, batch_size = 120)

Epoch 1/20
50000/50000 [==============================] - 8s 155us/step - loss: 1.6502 - acc: 0.4041
Epoch 2/20
50000/50000 [==============================] - 5s 106us/step - loss: 1.2341 - acc: 0.5608
Epoch 3/20
50000/50000 [==============================] - 5s 107us/step - loss: 1.0476 - acc: 0.6297
Epoch 4/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.9158 - acc: 0.6783
Epoch 5/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.8148 - acc: 0.7144
Epoch 6/20
50000/50000 [==============================] - 5s 106us/step - loss: 0.7209 - acc: 0.7494
Epoch 7/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.6426 - acc: 0.7761
Epoch 8/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.5749 - acc: 0.7992
Epoch 9/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.5192 - acc: 0.8197
Epoch 10/20
50000/50000 [==============================] - 5s 107us/step - loss: 0.4607 - a

In [6]:
vec_out = Model(inputs = inputs, outputs = model.layers[9].output)
preds = np.argmax(model.predict(x_test), axis = 1)
y_test = np.argmax(y_test, axis = 1)

print(np.mean(np.mean(preds==y_test)))

0.75


In [7]:
from sklearn.svm import SVC
X_train = vec_out.predict(x_train)
X_test = vec_out.predict(x_test)
linear = SVC()

In [ ]:
linear.fit(X_train, np.argmax(y_train, axis = 1))
pred = linear.predict(X_test)
pred = np.argmax(pred, axis=1)

In [ ]:
print(np.mean(np.mean(pred==y_test)))